In [36]:
import pickle
import pandas as pd
import sys
# sys.path.append("..")
# import dlp_model_evaluation
sys.path.append("/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/algorithm/build/cilqr")
sys.path.append("/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/algorithm/build/env_simulator")
import pybind_env_simulator
import pybind_ilqr
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource

# Initialize Bokeh to display in the notebook
output_notebook()


Loading BokehJS ...

In [37]:
agent_file_path = '/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/dlp_model_evaluation/test_data/output_5752de06-cbd1-42d8-8a81-03649a38636c/scenario_5752de06-cbd1-42d8-8a81-03649a38636c.parquet'
agent_data = pd.read_parquet(agent_file_path)
# print(agent_data)
# print(agent_data.columns)
if agent_data is not None:
    ego_data = agent_data[agent_data['track_id'] == 'AV']
    ego_origin_state = ego_data.iloc[49]
# print(ego_origin_state)

In [38]:
import csv
import math
import pickle
import argparse
import pandas as pd
import os
import json
import numpy  as np
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

class Point2D:
    def __init__(self, x=0.0, y=0.0):
        self.x = x
        self.y = y

class PlanningPoint:
    def __init__(self, position=None, theta=0.0, velocity=0.0, acceleration=0.0, 
                 omega=0.0, curva=0.0, jerk=0.0, omega_dot=0.0):
        self.position = position if position is not None else Point2D()
        self.theta = theta
        self.velocity = velocity
        self.acceleration = acceleration
        self.omega = omega
        self.curva = curva
        self.jerk = jerk
        self.omega_dot = omega_dot

    def __repr__(self):
        return (f"PlanningPoint(position=({self.position.x}, {self.position.y}), theta={self.theta}, "
                f"velocity={self.velocity}, acceleration={self.acceleration}, "
                f"omega={self.omega}, curva={self.curva}, jerk={self.jerk}, omega_dot={self.omega_dot})")

def read_json_file(json_file):
    map_data = None
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            map_data = json.load(file)
    except Exception as e:
        print(f"read map data failed")

    return map_data


def read_pkl_file(pkl_file):
    traj_data = None
    try:
        with open(pkl_file, 'rb') as file:
            traj_data = pickle.load(file)
    except Exception as e:
        print(f"read trajectory data failed")
    return traj_data


def inputs_preprocess(single_sample_dir, agent_file, map_file, pkl_file):
    map_file_path = os.path.join(single_sample_dir, map_file)
    agent_file_path = os.path.join(single_sample_dir, agent_file)
    pkl_file_path = os.path.join(single_sample_dir, pkl_file)
    map_data = read_json_file(map_file_path)
    agent_data = pd.read_parquet(agent_file_path)
    traj_data = read_pkl_file(pkl_file_path)
    ego_traj_data = None
    if traj_data is not None:
        # calculate v/s/heading/accel of prediction trajectory point via trajectory points position
        for obs_index, obs_trajs in traj_data['traj'].items():
            for each_traj in obs_trajs:
                sum_s = 0.0
                for point_index in range(len(each_traj)):
                    if point_index > 0:
                        sum_s += math.sqrt((each_traj[point_index]['x'] - each_traj[point_index - 1]['x']) ** 2 + (
                            each_traj[point_index]['y'] - each_traj[point_index - 1]['y']) ** 2)
                    each_traj[point_index]['s'] = round(sum_s, 2)

                    if point_index + 1 < len(each_traj):
                        dis_to_next_pt = math.sqrt(
                            (each_traj[point_index + 1]['x'] - each_traj[point_index]['x']) ** 2 + (
                                each_traj[point_index + 1]['y'] - each_traj[point_index]['y']) ** 2)
                        each_traj[point_index]['v'] = round(
                            dis_to_next_pt / 0.1, 2)
                        each_traj[point_index]['heading'] = math.atan2(
                            each_traj[point_index + 1]['y'] -
                            each_traj[point_index]['y'],
                            each_traj[point_index + 1]['x'] - each_traj[point_index]['x'])
                    elif len(each_traj) == 1:
                        each_traj[point_index]['v'] = 0.0
                        each_traj[point_index]['heading'] = 0.0
                    else:
                        each_traj[point_index]['v'] = round(
                            each_traj[point_index - 1]['v'], 2)
                        each_traj[point_index]['heading'] = round(
                            each_traj[point_index - 1]['heading'], 2)

                for point_index in range(len(each_traj)):
                    if point_index + 1 < len(each_traj):
                        each_traj[point_index]['accel'] = round(
                            (each_traj[point_index + 1]['v'] - each_traj[point_index]['v']) / 0.1, 2)
                    elif len(each_traj) == 1:
                        each_traj[point_index]['accel'] = 0.0
                    else:
                        each_traj[point_index]['accel'] = round(
                            each_traj[point_index - 1]['accel'], 2)

                for point_index in range(len(each_traj)):
                    if point_index + 1 < len(each_traj):
                        each_traj[point_index]['jerk'] = round(
                            (each_traj[point_index + 1]['accel'] - each_traj[point_index]['accel']) / 0.1, 2)
                    elif len(each_traj) == 1:
                        each_traj[point_index]['jerk'] = 0.0
                    else:
                        each_traj[point_index]['jerk'] = round(
                            each_traj[point_index - 1]['jerk'], 2)

                for point_index in range(len(each_traj)):
                    if point_index + 1 < len(each_traj):
                        diff_theta = each_traj[point_index +
                                               1]['heading'] - each_traj[point_index]['heading']
                        if abs(diff_theta) > math.pi:
                            each_traj[point_index]['yawrate'] = round((diff_theta - 2 * math.pi) / 0.1,
                                                                      2) if math.copysign(1,
                                                                                          diff_theta) > 0.0 else round(
                                (-2 * math.pi - diff_theta) / 0.1, 2)
                        else:
                            each_traj[point_index]['yawrate'] = round(
                                (each_traj[point_index + 1]['heading'] - each_traj[point_index]['heading']) / 0.1, 2)
                    elif len(each_traj) == 1:
                        each_traj[point_index]['yawrate'] = 0.0
                    else:
                        each_traj[point_index]['yawrate'] = round(
                            each_traj[point_index - 1]['yawrate'], 2)
    # get ego origin state
    ego_origin_state = PlanningPoint()
    if agent_data is not None:
        ego_data = agent_data[(agent_data['track_id'] == 'AV') & (
            agent_data['timestep'] == 50)]
        if not ego_data.empty:
            ego_origin_state.position.x = ego_data.iloc[0]['position_x']
            ego_origin_state.position.y = ego_data.iloc[0]['position_y']
            ego_origin_state.velocity = math.sqrt(
                ego_data.iloc[0]['velocity_x'] ** 2 +
                ego_data.iloc[0]['velocity_y'] ** 2
            )
            ego_origin_state.theta = ego_data.iloc[0]['heading']
        else:
            print("Error: No AV data available at timestep 50.")
    # get ego traj
    sum_error = []
    for obs_index, trajs in traj_data['traj'].items():
        error_dis = 0.0
        for each_traj in trajs:
            first_pt = each_traj[0]
            error_dis += math.sqrt((first_pt['x'] - ego_origin_state.position.x) ** 2 +
                                   (first_pt['y'] - ego_origin_state.position.y) ** 2)
        sum_error.append(error_dis)
    ego_traj_index = sum_error.index(min(sum_error))
    ego_row = traj_data['pi'][ego_traj_index]
    max_probability_ego_traj_index = np.argmax(ego_row)
    ego_traj_data = traj_data['traj'][ego_traj_index][max_probability_ego_traj_index]

    return map_data, agent_data, ego_traj_data, ego_origin_state



In [39]:

class CallILQR:
    def __init__(self, ref_pts, ego_origin):
        self.ref_pts_ = ref_pts
        self.ego_origin_ = ego_origin
        self.processed_trajs_ = []
        self.motion_ = pybind_ilqr.LateralLongitudinalMotion()
        # for plot test
        self.plot_data_ = {}

    def process(self):
        point_init = pybind_ilqr.PlanningPoint()
        point_init.position.x = self.ego_origin_.position.x
        point_init.position.y = self.ego_origin_.position.y
        point_init.theta = self.ego_origin_.theta
        point_init.velocity = self.ego_origin_.velocity
        point_init.acceleration = 0.0

        trajectory_points = []
        for pt in self.ref_pts_:
            point = pybind_env_simulator.Point2D()
            point.x = pt['x']
            point.y = pt['y']
            trajectory_points.append(point)
        self.motion_.init(trajectory_points, point_init)
        self.motion_.execute_tree()
        return

    def get_processed_trajs(self):
        traj_trees = self.motion_.get_trajectory_tree()
        print("trajectory tree branch size is: ", len(traj_trees))
        lon_debug = self.motion_.get_lon_debug_tree()
        lat_debug = self.motion_.get_lat_debug_tree()
        self.processed_trajs_ = []
        j = 0
        for traj_tree in traj_trees:
            traj_new_list = []
            index = 0
            for traj in traj_tree:
                pt = {}
                pt['x'] = traj.position.x
                pt['y'] = traj.position.y
                pt['theta'] = traj.theta
                pt['v'] = traj.velocity
                pt['acc'] = traj.acceleration
                pt['acc_lat'] = traj.velocity * traj.omega
                pt['ref_vel'] = lon_debug[j][index].v_ref
                pt['ref_acc'] = lon_debug[j][index].a_ref
                pt['match_point'] = lat_debug[j][index].match_point
                pt['jerk'] = traj.jerk
                pt['omega_dot'] = traj.omega_dot
                index = index + 1
                traj_new_list.append(pt)
            self.processed_trajs_.append(traj_new_list)
            j = j + 1
        return self.processed_trajs_

    def get_plot_data(self):
        # generate plot data
        env = self.motion_.get_env()
        planning_origin = self.motion_.get_planning_origin()
        pts = env.get_all_center_points()
        obs = env.get_all_obstacle()
        planning_init = {}
        planning_init['x'] = planning_origin.position.x
        planning_init['y'] = planning_origin.position.y
        planning_init['theta'] = planning_origin.theta
        trajs = self.motion_.get_init_trajectory()
        traj_list = []
        for traj in trajs:
            pt = {}
            pt['x'] = traj.position.x
            pt['y'] = traj.position.y
            pt['theta'] = traj.theta
            traj_list.append(pt)
        self.plot_data_['pts'] = pts
        self.plot_data_['obs'] = obs
        self.plot_data_['planning_init'] = planning_init
        self.plot_data_['traj_list'] = traj_list
        return self.plot_data_

    def get_ref_pts(self):
        return self.ref_pts_


In [40]:
def convert_type(processed_trajectory, size:int):
    ans = []
    for traj in processed_trajectory:
        sum_s = 0.0
        for point_index in range(len(traj) - 1):
            pt_left = traj[point_index]
            pt_right = traj[point_index + 1]
            
            if point_index > 0:
                sum_s += math.sqrt(
                    (traj[point_index]['x'] - traj[point_index - 1]['x']) ** 2 + 
                    (traj[point_index]['y'] - traj[point_index - 1]['y']) ** 2
                )
            
            pt_left_data = {
                'x': pt_left['x'],
                'y': pt_left['y'],
                's': round(sum_s, 2),
                'v': pt_left['v'],
                'heading': pt_left['theta'],
                'accel': pt_left['acc'],
                'jerk': pt_left['jerk'],
                'yawrate': pt_left['omega_dot']
            }
            ans.append(pt_left_data)
            
            pt_mid = {
                'x': (pt_left['x'] + pt_right['x']) / 2,
                'y': (pt_left['y'] + pt_right['y']) / 2,
                's': round(sum_s + math.sqrt(
                    (pt_right['x'] - pt_left['x']) ** 2 + 
                    (pt_right['y'] - pt_left['y']) ** 2) / 2, 2),
                'v': (pt_left['v'] + pt_right['v']) / 2,
                'heading': (pt_left['theta'] + pt_right['theta']) / 2,
                'accel': (pt_left['acc'] + pt_right['acc']) / 2,
                'jerk': (pt_left['jerk'] + pt_right['jerk']) / 2,
                'yawrate': (pt_left['omega_dot'] + pt_right['omega_dot']) / 2
            }
            ans.append(pt_mid)
        
        # last_point = traj[-1]
        # sum_s += math.sqrt(
        #     (last_point['x'] - traj[-2]['x']) ** 2 + 
        #     (last_point['y'] - traj[-2]['y']) ** 2
        # )
        # pt_last = {
        #     'x': last_point['x'],
        #     'y': last_point['y'],
        #     's': round(sum_s, 2),
        #     'v': last_point['v'],
        #     'heading': last_point['theta'],
        #     'accel': last_point['acc'],
        #     'jerk': last_point['jerk'],
        #     'yawrate': last_point['omega_dot']
        # }
        # ans.append(pt_last)

    return ans

In [49]:

single_sample_dir = '/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/dlp_model_evaluation/test_data/output_5752de06-cbd1-42d8-8a81-03649a38636c'
agent_file = '/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/dlp_model_evaluation/test_data/output_5752de06-cbd1-42d8-8a81-03649a38636c/scenario_5752de06-cbd1-42d8-8a81-03649a38636c.parquet'
map_file = '/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/dlp_model_evaluation/test_data/output_5752de06-cbd1-42d8-8a81-03649a38636c/log_map_archive_5752de06-cbd1-42d8-8a81-03649a38636c.json'
pkl_file = '/home/SENSETIME/gaoyuhui/ws/pilotmdc_rscl/senseauto-pilot-decision/common_math/athena/dlp_model_evaluation/test_data/output_5752de06-cbd1-42d8-8a81-03649a38636c/output_batch_0.pkl'

map_data, agent_data, ego_traj_data, ego_origin_state = inputs_preprocess(
    single_sample_dir, agent_file, map_file, pkl_file)

print(len(ego_traj_data))

call_ilqr = CallILQR(ego_traj_data, ego_origin_state)
call_ilqr.process()
processed_trajectory = call_ilqr.get_processed_trajs()
ilqr_ego_traj_data = convert_type(processed_trajectory, 60)
print(len(ilqr_ego_traj_data))


# Extracting data from both ego_traj_data and the new dataset
index = list(range(len(ego_traj_data)))
traj_x, traj_y, traj_v, traj_s = [], [], [], []
traj_heading, traj_accel, traj_jerk, traj_yawrate = [], [], [], []

for pt in ego_traj_data:
    traj_x.append(pt['x'])
    traj_y.append(pt['y'])
    traj_v.append(pt['v'])
    traj_s.append(pt['s'])
    traj_heading.append(pt['heading'])
    traj_accel.append(pt['accel'])
    traj_jerk.append(pt['jerk'])
    traj_yawrate.append(pt['yawrate'])

# Data for the second dataset
index_2 = list(range(len(ilqr_ego_traj_data)))
traj_x2, traj_y2, traj_v2, traj_s2 = [], [], [], []
traj_heading2, traj_accel2, traj_jerk2, traj_yawrate2 = [], [], [], []

for pt in ilqr_ego_traj_data:
    traj_x2.append(pt['x'])
    traj_y2.append(pt['y'])
    traj_v2.append(pt['v'])
    traj_s2.append(pt['s'])
    traj_heading2.append(pt['heading'])
    traj_accel2.append(pt['accel'])
    traj_jerk2.append(pt['jerk'])
    traj_yawrate2.append(pt['yawrate'])

# XY Scatter plot with both datasets
source_xy = ColumnDataSource(data=dict(x=traj_x, y=traj_y))
source_xy2 = ColumnDataSource(data=dict(x=traj_x2, y=traj_y2))

p_xy = figure(width=500, height=450, title="XY Coordinate Points")
p_xy.scatter('x', 'y', source=source_xy, size=8, color="navy", alpha=0.6, legend_label="raw traj")
p_xy.scatter('x', 'y', source=source_xy2, size=8, color="green", alpha=0.6, legend_label="ilqr traj")

# Plot template function with two datasets
def create_dual_line_plot(index1, values1, index2, values2, title, color1="blue", color2="green"):
    source1 = ColumnDataSource(data=dict(index=index1, value=values1))
    source2 = ColumnDataSource(data=dict(index=index2, value=values2))
    
    p = figure(title=title, x_axis_label="Index", y_axis_label="Value", width=500, height=150)
    p.line('index', 'value', source=source1, line_width=2, color=color1, alpha=0.6, legend_label="raw")
    p.scatter('index', 'value', source=source1, size=8, color=color1, alpha=0.6)
    
    p.line('index', 'value', source=source2, line_width=2, color=color2, alpha=0.6, legend_label="ilqr")
    p.scatter('index', 'value', source=source2, size=8, color=color2, alpha=0.6)
    
    return p

# Creating individual plots with both datasets
p_v = create_dual_line_plot(index, traj_v, index_2, traj_v2, "Velocity (v)")
p_s = create_dual_line_plot(index, traj_s, index_2, traj_s2, "Distance (s)")
p_heading = create_dual_line_plot(index, traj_heading, index_2, traj_heading2, "Heading")
p_accel = create_dual_line_plot(index, traj_accel, index_2, traj_accel2, "Acceleration")
p_jerk = create_dual_line_plot(index, traj_jerk, index_2, traj_jerk2, "Jerk")
p_yawrate = create_dual_line_plot(index, traj_yawrate, index_2, traj_yawrate2, "Yaw Rate")

# Arrange plots in a grid layout
col1 = column(p_v, p_s, p_heading)
col2 = column(p_accel, p_jerk, p_yawrate)
layout = row(p_xy, col1, col2)

# Display the layout
show(layout)





60
 r_4 is :  
1.13033 
trajectory tree branch size is:  1
60
j_opt: 8141.347141 
--- stage 0 --- reach max iter, update to next stage ---  
--- stage 1 --- succeed! next state ---  
---iter times: 4 
---succeed! stop search--- 
 **************** ilqr tree time: 44.6272 ms 
 tree trajectory finish


trajectory tree branch size is:  1
60
 r_4 is :  
1.13033 
j_opt: 8141.347141 
--- stage 0 --- reach max iter, update to next stage ---  
--- stage 1 --- succeed! next state ---  
---iter times: 4 
---succeed! stop search--- 
 **************** ilqr tree time: 47.3062 ms 
 tree trajectory finish
